In [ ]:
# Created by Jungsun Yoo on 2020-01-09
# define UNet Model in TF
import tensorflow as tf

def weight_variable(shape, name):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides = [1, 2, 2, 1], padding = 'SAME')

# tf.nn.conv2d(input, filters, strides, padding, ...)
# Given an input tensor of shape [batch, in_height, in_width, in_channels] (=x shape) and a filter
# / kernel tensor of shape [filter_height, filter_width, in_channels, out_channels], this op performs the following:
# 1. Flattens the filter to a 2D matrix with shape [filter_height * filter_width * in_channels, output_channels]
# 2. Extracts image patches from the input tensor to form a virtual tensor of shape [batch, output_height, output_width,
# filter_height * filter_width * in_channels]
# 3. For each patch, right-multiplies the filter matrix and the image patch vector
# Must have strides[0] = strides[3] = 1 

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1])


def upsample_simple(x, [new_height, new_width]):
    # it's height, width in TF - not width, height
    # new_height = int(round(old_height * scale))
    # new_width = int(round(old_width * scale))
    # resized = tf.image.resize_images(input_tensor, [new_height, new_width])
    return tf.image.resize_images(x, [new_height, new_width])

In [6]:
def model_tf(x, base, scale):  #base default = 32; scale default = 2
    #===== first layer
    W_conv1 = weight_variable([3, 3, 1, base], "W_conv1") # filter/kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
    b_conv1 = bias_variable([base], "b_conv1")

    x_image = tf.reshape(x, [-1, 256, 256, 1]) # [batch, in_height, in_width, in_channels]
    
    h_conv1= tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
    
    W_conv2 = weight_variable([3, 3, base, base], "W_conv2")
    b_conv2 = bias_variable([base], "b_conv2")
    
    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
    
    h_pool1 = max_pool_2x2(h_conv2)
    
    #===== second layer    
    W_conv3 = weight_variable([3,3,base, (scale)*base], "W_conv3")
    b_conv3 = bias_variable([(scale)*base], "b_conv3")
    
    h_conv3 = tf.nn.relu(conv2d(h_pool1, W_conv3) + b_conv3)
    
    W_conv4 = weight_variable([3,3,(scale)*base,(scale)*base], "W_conv4")
    b_conv4 = bias_variable([(scale)*base], "b_conv4")

    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4) + b_conv4)

    h_pool2 = max_pool_2x2(h_conv4)
    
    #===== third layer    
    W_conv5 = weight_variable([3,3,(scale)*base, (scale * scale) * base], "W_conv5")
    b_conv5 = bias_variable([(scale * scale) * base], "b_conv5")
    
    h_conv5 = tf.nn.relu(conv2d(h_pool2, W_conv5) + b_conv5)
    
    W_conv6 = weight_variable([3,3,(scale * scale) * base,(scale * scale) * base], "W_conv6")
    b_conv6 = bias_variable([(scale * scale) * base], "b_conv6")

    h_conv6 = tf.nn.relu(conv2d(h_conv5, W_conv6) + b_conv6)

    h_pool3 = max_pool_2x2(h_conv6)   
    
    #===== fourth layer    
    W_convc1 = weight_variable([3,3,(scale * scale)*base, (scale * scale * scale) * base], "W_convc1")
    b_convc1 = bias_variable([(scale * scale * scale) * base], "b_convc1")
    
    h_convc1 = tf.nn.relu(conv2d(h_pool3, W_convc1) + b_convc1)
    
    W_convc2 = weight_variable([3,3,(scale * scale * scale) * base,(scale * scale * scale) * base], "W_convc2")
    b_convc2 = bias_variable([(scale * scale * scale) * base], "b_convc2")

    h_convc2 = tf.nn.relu(conv2d(h_convc1, W_convc2) + b_convc2)
    
    #====== fifth layer = start upsampling
    # 1. Upsample and merge
    
    h_pool4 = upsample_simple(h_convc2, [(scale * scale)*base, (scale * scale)*base])
    merge = concatenate([h_pool4, h_conv6])
    
    W_conv7 = weight_variable([3, 3, (scale*scale*scale)*base, (scale*scale)*base], "W_conv7")
    b_conv7 = bias_variable([(scale*scale)*base], "b_conv7")
    
    h_conv7 = tf.nn.relu(conv2d(merge, W_conv7) + b_conv7)
    
    W_conv8 = weight_variable([3, 3, (scale*scale)*base, (scale*scale)*base], "W_conv8")
    b_conv8 = bias_variable([(scale*scale)*base], "b_conv8")
    
    h_conv8 = tf.nn.relu(conv2d(conv7, W_conv8) + b_conv8)
    
    # ---- sixth layer = upsample2
    
    h_pool5 = upsample_simple(h_conv8, [(scale)*base, (scale)*base])
    merge = concatenate([h_pool5, h_conv4])
    
    W_conv9 = weight_variable([3, 3, (scale*scale)*base, (scale)*base], "W_conv9")
    b_conv9 = bias_variable([(scale)*base], "b_conv9")
    
    h_conv9 = tf.nn.relu(conv2d(merge, W_conv9) + b_conv9)
    
    W_conv10 = weight_variable([3, 3, (scale)*base, (scale)*base], "W_conv10")
    b_conv10 = bias_variable([(scale)*base], "b_conv10")
    
    h_conv10 = tf.nn.relu(conv2d(h_conv9, W_conv10) + b_conv10)  
    
    # ------
    
    h_pool6 = upsample_simple(h_conv10, [base, base])
    merge = concatenate([h_pool6, h_conv2])
    
    W_conv11 = weight_variable([3, 3, (scale)*base, base], "W_conv11")
    b_conv11 = bias_variable([base], "b_conv11")
    
    h_conv11 = tf.nn.relu(conv2d(merge, W_conv11) + b_conv11)
    
    W_conv12 = weight_variable([3, 3, base, base], "W_conv12")
    b_conv12 = bias_variable([base], "b_conv12")
    
    
#     out = tf.nn.softmax(conv2d(h_conv11, W_conv12) + b_conv12)
    y_conv = tf.nn.softmax(tf.matmul(h_conv11, W_conv12) + b_conv12)
#     variable_dict = {"W_conv1": W_conv1, "b_conv1": b_conv1, "W_conv2": W_conv2, "b_conv2": b_conv2, "W_conv3": W_conv3, 
#                     "b_conv3": b_conv3, }
    
    return y_conv, variable_dict
    
#     y_conv = tf.nn.softmax(tf.matmul(merge, W_conv11) + b_conv11)
    
    
    
    

In [7]:
a = model_tf

In [8]:
a.summary()

AttributeError: 'function' object has no attribute 'summary'